<a href="https://colab.research.google.com/github/shobhit115/HackIndia-Spark-9-2025-Code-Hackers/blob/main/Lawgeni_Data_cleanning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load both CSV files
csv1 = pd.read_csv('/content/direct_pdfs_found.csv', header=None)
csv2 = pd.read_csv('/content/scraped_pdfs_found.csv', header=None)

# Extract link columns (first col in csv1, second col in csv2)
links1 = csv1.iloc[:, 0]
links2 = csv2.iloc[:, 1]

# Combine all links into one list
merged_links = pd.concat([links1, links2], ignore_index=True)

# Remove duplicates and empty values
merged_links = merged_links.dropna().drop_duplicates().reset_index(drop=True)

# Save to new CSV
merged_links.to_csv('merged_links.csv', index=False, header=['pdf_link'])

print(f"Merged {len(merged_links)} unique links saved to 'merged_links.csv'")


Merged 5530 unique links saved to 'merged_links.csv'


In [3]:
!pip install PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 92.8 MB/s eta 0:00:00


In [4]:
import fitz  # PyMuPDF

def extract_text_from_pdf(path):
    text = ""
    with fitz.open(path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    return text.strip()


In [6]:
!pip install pytesseract

In [7]:
import fitz
import pytesseract
from PIL import Image
import io

def extract_text_from_scanned_pdf(path):
    text = ""
    with fitz.open(path) as doc:
        for page in doc:
            pix = page.get_pixmap()
            img = Image.open(io.BytesIO(pix.tobytes()))
            text += pytesseract.image_to_string(img)
    return text.strip()


In [8]:
def smart_extract(path):
    text = extract_text_from_pdf(path)
    if len(text.strip()) < 50:  # likely scanned
        text = extract_text_from_scanned_pdf(path)
    return text


In [31]:
import re

def structure_legal_text(text):
    # Normalize spaces
    cleaned_text = re.sub(r'\s+', ' ', text).strip()

    # --- Title Extraction ---
    title_pattern = re.compile(
        r'(?:THE\s+[A-Z\s\(\)\-]+ACT(?:,?\s*\d{4})?|ACT\s+NO\.?\s*[A-Z0-9\-]*\.?\s*OF\s*\d{4})',
        re.IGNORECASE
    )
    title_match = title_pattern.search(cleaned_text)
    title = title_match.group(0).strip() if title_match else None

    # --- Act Year Extraction ---
    year_match = re.search(r'(?:OF|,)\s*(\d{4})', cleaned_text, re.IGNORECASE)
    act_year = int(year_match.group(1)) if year_match else None

    # --- Section / Clause Detection ---

    sections = re.findall(
        r'\b(?:Section\s+\d+[A-Za-z]?|\b\d{1,3}\.\s+[A-Z]|\b[IVXLCDM]{1,6}\.)',
        cleaned_text
    )

    structured = {
        "title": title,
        "act_year": act_year,
        "section_count": len(sections),
        "text": cleaned_text
    }

    return structured


In [32]:
#testing purpose
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import os

# ---- Import your previous functions ----
# from your_module import smart_extract, structure_legal_text

# Load the merged CSV containing PDF links
df = pd.read_csv('merged_links.csv')
pdf_links = df['pdf_link']

# Take first 10 for testing
test_links = pdf_links.head(10)

# Create folders
pdf_dir = Path("pdfs")
pdf_dir.mkdir(exist_ok=True)

output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

# Storage for structured output
output = []

# ---- Process each PDF ----
for i, link in enumerate(tqdm(test_links, desc="Processing PDFs", unit="pdf")):
    pdf_name = Path(link).name
    pdf_path = pdf_dir / pdf_name

    # Step 1: Download PDF if not already present
    if not pdf_path.exists():
        os.system(f"curl -L -o '{pdf_path}' '{link}'")

    # Step 2: Extract text (auto-detects scanned vs normal)
    try:
        text = smart_extract(pdf_path)
    except Exception as e:
        print(f"[Error] {pdf_name}: {e}")
        continue

    # Step 3: Structure the extracted text
    data = structure_legal_text(text)
    data["file_name"] = pdf_name
    data["pdf_link"] = link  # ✅ Add the source PDF link

    output.append(data)

# ---- Save results ----
output_df = pd.DataFrame(output)
output_path = output_dir / 'structured_output_test.csv'
output_df.to_csv(output_path, index=False)

print(f"\n✅ Processed {len(output)} PDFs successfully.")
print(f"Output saved to {output_path}")


Processing PDFs:  30%|███       | 3/10 [00:00<00:00, 19.87pdf/s]

[Error] direct_pdf_link: no such file: 'pdfs/direct_pdf_link'


Processing PDFs: 100%|██████████| 10/10 [00:00<00:00, 22.62pdf/s]


✅ Processed 9 PDFs successfully.
Output saved to outputs/structured_output_test.csv


In [33]:
#testing purpose
import pandas as pd

df = pd.read_csv("outputs/structured_output_test.csv")
df.head(10).to_json("outputs/preview.json", orient="records", indent=4)


In [ ]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import os
import math

# ---- Import your helper functions ----
# from your_module import smart_extract, structure_legal_text

# Load merged CSV
df = pd.read_csv('merged_links.csv')
pdf_links = df['pdf_link'].dropna().tolist()

# Create folders
pdf_dir = Path("pdfs")
pdf_dir.mkdir(exist_ok=True)

output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

# Batch size
BATCH_SIZE = 100
num_batches = math.ceil(len(pdf_links) / BATCH_SIZE)

print(f"Total PDFs: {len(pdf_links)} → {num_batches} batches of {BATCH_SIZE} each\n")

# ---- Process in batches ----
for batch_num in range(num_batches):
    start_idx = batch_num * BATCH_SIZE
    end_idx = min(start_idx + BATCH_SIZE, len(pdf_links))
    batch_links = pdf_links[start_idx:end_idx]

    print(f"🚀 Processing batch {batch_num + 1}/{num_batches} ({start_idx + 1}-{end_idx})")

    output = []

    for link in tqdm(batch_links, desc=f"Batch {batch_num + 1}", unit="pdf"):
        pdf_name = Path(link).name
        pdf_path = pdf_dir / pdf_name

        # Step 1: Download if not already present
        if not pdf_path.exists():
            os.system(f"curl -s -L -o '{pdf_path}' '{link}'")

        # Step 2: Extract text
        try:
            text = smart_extract(pdf_path)
        except Exception as e:
            print(f"[Error] {pdf_name}: {e}")
            continue

        # Step 3: Structure
        try:
            data = structure_legal_text(text)
            data["file_name"] = pdf_name
            data["pdf_link"] = link
            output.append(data)
        except Exception as e:
            print(f"[Error structuring] {pdf_name}: {e}")
            continue

    # Step 4: Save this batch’s output
    batch_output = pd.DataFrame(output)
    output_file = output_dir / f'structured_output_batch_{batch_num + 1}.csv'
    batch_output.to_csv(output_file, index=False)

    print(f"✅ Batch {batch_num + 1} done → saved to {output_file}\n")

print("🎯 All batches processed successfully.")


Total PDFs: 5530 → 56 batches of 100 each

🚀 Processing batch 1/56 (1-100)


Batch 1:   3%|▎         | 3/100 [00:00<00:05, 18.26pdf/s]

[Error] direct_pdf_link: no such file: 'pdfs/direct_pdf_link'


Batch 1: 100%|██████████| 100/100 [02:11<00:00,  1.32s/pdf]


[Error] repealedfileopen?rfilename=A1838-30.pdf: Cannot open empty file: filename='pdfs/repealedfileopen?rfilename=A1838-30.pdf'.
✅ Batch 1 done → saved to outputs/structured_output_batch_1.csv

🚀 Processing batch 2/56 (101-200)


Batch 2: 100%|██████████| 100/100 [05:22<00:00,  3.22s/pdf]


✅ Batch 2 done → saved to outputs/structured_output_batch_2.csv

🚀 Processing batch 3/56 (201-300)


Batch 3: 100%|██████████| 100/100 [07:55<00:00,  4.76s/pdf]


✅ Batch 3 done → saved to outputs/structured_output_batch_3.csv

🚀 Processing batch 4/56 (301-400)


Batch 4:  64%|██████▍   | 64/100 [05:06<07:23, 12.31s/pdf]